<a href="https://colab.research.google.com/github/julienif/julifsChessBot/blob/main/julifsChessBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip3 install torchrl
!pip3 install gym[mujoco]
!pip3 install gymnasium
!pip3 install tqdm
!pip3 install chess
!pip install torch torchvision torchaudio
!pip3 install tensorflow

Traceback (most recent call last):
  File "<frozen importlib._bootstrap_external>", line 1372, in _path_importer_cache


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 77, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/__init__.py", line 114, in create_command
    module = importlib.import_module(module_path)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen 

In [21]:
# imports

In [22]:
import warnings
warnings.filterwarnings("ignore")
from torch import multiprocessing

import numpy as np
import chess
import torch
import tensorflow as tf
import gymnasium as gym
from torch import nn
from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor
from gym import spaces


In [23]:
# chess RL environment

In [24]:
class ChessEnv(gym.Env):
  def __init__(self, color_trained='white', device='cpu'):
    super(ChessEnv, self).__init__()
    self.device = device
    self.color_trained = color_trained
    self.board = chess.Board()
    self.action_space = spaces.Discrete(4096) # assume max 4096 legal moves
    self.observation_space = spaces.Box(low=0, high=1, shape=(8, 8, 13))
    self.reward_range = (-1, 1)

  def reset(self):
    self.board.reset()
    return self._get_observation, {}

  def step(self, action):
    self.board.push(chess.Move.from_uci(action))

    obs = self._get_observation()
    reward = self._get_reward()
    terminated = self.board.is_game_over()
    truncated = False
    info = {}

    return obs, reward, terminated, truncated, info

  def render(self):
    print(self.board)
    return None

  def close(self):
    pass

  def _get_observation(self):
    board_state = np.zeros((8, 8, 13), dtype=np.float32)

    piece_map = {
      'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5,
      'P': 6, 'N': 7, 'B': 8, 'R': 9, 'Q': 10, 'K': 11,
      '.': 12
    }

    for square in chess.SQUARES:
      piece = self.board.piece_at(square)
      if piece is not None:
        idx = piece_map[piece.symbol()]
        board_state[chess.square_rank(square), chess.square_file(square), idx] = 1
      else:
        board_state[chess.square_rank(square), chess.square_file(square), 12] = 1

    return torch.tensor(board_state).to(self.device)

  def _get_reward(self):
    # might change this as we would have a constant negative reward if the bot is lead in the game
    reward = 0.0
    if self.board.is_checkmate():
      if self.board.turn == chess.WHITE: # black won
        reward = -1.0
      else: # white won
        reward = 1.0
    elif self.board.is_stalemate() or self.board.is_insufficient_material() or self.board.is_seventyfive_moves():
      reward = 0.0
    else:
      white_score = sum([piece.value for piece in self.board.piece_map().values() if piece.color == chess.WHITE])
      black_score = sum([piece.value for piece in self.board.piece_map().values() if piece.color == chess.BLACK])

      material_adv = white_score - black_score
      norm_material_adv = material_adv / 39.0 # 39 is the max mat diff

      total_moves = len(list(self.board.move_stack))
      progress = min(total_moves / 150.0, 1.0) # 150 moves is the average game

      reward = 0.7 * norm_material_adv + 0.3 * progress

    return reward if self.color_trained == 'white' else -reward

In [25]:
# implement training using PPO

In [29]:
# define hyper parameters

is_fork = multiprocessing.get_start_method() == "fork"
device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)
print(device)
num_cells = 256  # number of cells in each layer i.e. output dim.
lr = 3e-4
max_grad_norm = 1.0

frames_per_batch = 1000
# For a complete training, bring the number of frames up to 1M
total_frames = 50_000

sub_batch_size = 64  # cardinality of the sub-samples gathered from the current data in the inner loop
num_epochs = 10  # optimization steps per batch of data collected
clip_epsilon = (
    0.2  # clip value for PPO loss
)
gamma = 0.99
lmbda = 0.95
entropy_eps = 1e-4

# init environment
chess_env = ChessEnv(color_trained='white', device=device)























































GPU device name: /device:GPU:0
cuda:0
